In [29]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/Users/poraya/AI-project/train.csv'):
     for filename in filenames:
        print(os.path.join(dirname, filename))


In [30]:
# Import Libraries, classes, and functions 
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler 
from sklearn.impute import KNNImputer
from sklearn.feature_selection import mutual_info_regression 
from sklearn.decomposition import PCA, TruncatedSVD  
from sklearn.compose import make_column_transformer 
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, KFold, StratifiedKFold, StratifiedShuffleSplit  
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier  
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.naive_bayes import GaussianNB 
from sklearn.neural_network import MLPClassifier 
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score 


# Set random seed 
SEED = np.random.default_rng().integers(99999)
# SEED = 999999 
from tensorflow.keras.utils import set_random_seed 
set_random_seed = SEED

Step1.1 Import train.csv 

In [28]:
# Import training dataset into padas DataFrame & explore 
train = pd.read_csv('/Users/poraya/Documents/AI-project/train.csv') 

print(train.head(), '\n', 
train.info(), '\n', 
train.describe(include='all').T, '\n', 
train.nunique() , '\n', 
train.isna().sum() )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB
  PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   


Step1.2 Import test.csv 

In [8]:
# Import test dataset into padas DataFrame & explore 
test = pd.read_csv('/Users/poraya/Documents/AI-project/test.csv') 

print(test.head(), '\n', 
test.info(), '\n', 
test.describe(include='all').T, '\n', 
test.nunique() , '\n', 
test.isna().sum() )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB
  PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0013_01      Earth      True  G/3/S  TRAPPIST-1e  27.0  False   
1     0018_01      Earth     False  F/4/S  TRAPPIST-

Step1.3 Merge train and test

In [9]:
# merge test and train datasets into single dataframe for data cleaning and necessary pre-processing 
all = pd.concat([train, test], axis=0) 

print(all.head(), '\n', 
all.info(), '\n', 
all.describe(include='all').T, '\n', 
all.nunique() , '\n', 
all.isna().sum() )

<class 'pandas.core.frame.DataFrame'>
Index: 12970 entries, 0 to 4276
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   12970 non-null  object 
 1   HomePlanet    12682 non-null  object 
 2   CryoSleep     12660 non-null  object 
 3   Cabin         12671 non-null  object 
 4   Destination   12696 non-null  object 
 5   Age           12700 non-null  float64
 6   VIP           12674 non-null  object 
 7   RoomService   12707 non-null  float64
 8   FoodCourt     12681 non-null  float64
 9   ShoppingMall  12664 non-null  float64
 10  Spa           12686 non-null  float64
 11  VRDeck        12702 non-null  float64
 12  Name          12676 non-null  object 
 13  Transported   8693 non-null   object 
dtypes: float64(6), object(8)
memory usage: 1.5+ MB
  PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002_01  

In [10]:
# drop duplicate rows 
all.drop_duplicates(keep='first', inplace=True) 
print(all.dtypes)

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported      object
dtype: object


In [11]:
# Feature Engineering 
# split PassengerId column into its component parts (group, and id ) as separate features  
all[['group', 'id']] = all['PassengerId'].str.split(pat='_', expand=True) 

# split Cabin column into its component parts (deck, cabin id, side ) as separate features 
all[['deck', 'cabin_no', 'side']] = all.Cabin.str.split('/', expand=True) 

# split Name column into its component parts (first and last name) as separate features
all[['FName', 'LName']] = all.Name.str.split(" ", expand=True) 
''' The intuition behind splitting name is that family name or first name may potentially 
have some influence on who gets transported into the anomaly '''

# Treatment of High-cardinality columns : columns that have too many categories (eg. upwards of 25 categories) 
''' add value_counts column for high cardinality columns & then drop original col, and treat resulting values as numerical data 
alternatively, transform high_cardinality column data into numbers (ordinal encoding) and treat as numerical data '''

# drop unique value columns  
print(all.columns)

# Subsetting columns for pre-processing with relevant encoders 
''' Since most machine learning models will require all data in numerical form, we will use relevant 
encoders to transform our data accordingly. Numerical columns will not undergo any encoding. 
Columns with ordinal values (ranks, value_counts, high-cardinality columns, etc) will be processed 
using Ordinal Encoder. Columns with categorical values (low-cardinality unique un-ordered values) 
to be OneHotEncoded, but we will first convert them into numerical codes (with Ordinal Encoder) so 
the whole dataset is already numerical for missing value imputation using an Imputation algorithms, 
eg. KNNImputer we have used here. '''

numeric = ['Age', 'group', 'id', 'cabin_no', 'RoomService', 'FoodCourt', 'ShoppingMall','Spa','VRDeck',] 
ordinal = ['FName', 'LName'] 
categorical = ['HomePlanet', 'Destination', 'deck', 'side', 'CryoSleep', 'VIP', 'Transported'] 

# checking unique values in each categorical columns to address 
# any mistakes (such as due to spelling, uppercase-lowercase or similar errors,)
uniq_cats = {col: sorted(all[col].dropna().unique().tolist() ) for col in categorical} 
# print(uniq_cats) 

# Converting target column booleans to corresponding Integer values 
all.Transported = all.Transported.replace({True: 1, False:0})

# Viewing summary statistics for all columns to check for any un-natural (eg. negative values where there should be none, etc) values,
print(all.describe(include='all').T)

# converting into numeric dtypes where numbers are appearing as object datatype 
all[['group', 'id', 'cabin_no']] = all[['group', 'id', 'cabin_no']].apply(pd.to_numeric, axis=0)
print(all.info())

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported', 'group', 'id', 'deck', 'cabin_no', 'side',
       'FName', 'LName'],
      dtype='object')
                count unique               top   freq        mean  \
PassengerId     12970  12970           0001_01      1         NaN   
HomePlanet      12682      3             Earth   6865         NaN   
CryoSleep       12660      2             False   8079         NaN   
Cabin           12671   9825           G/734/S      8         NaN   
Destination     12696      3       TRAPPIST-1e   8871         NaN   
Age           12700.0    NaN               NaN    NaN   28.771969   
VIP             12674      2             False  12401         NaN   
RoomService   12707.0    NaN               NaN    NaN  222.897852   
FoodCourt     12681.0    NaN               NaN    NaN  451.961675   
ShoppingMall  12664.0    NaN      

/var/folders/gr/pfv5w5hx4m904t97xs_9qv7m0000gn/T/ipykernel_84124/2749992746.py:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  all.Transported = all.Transported.replace({True: 1, False:0})


In [12]:
# dropping columns that have been split into their component values 
# setting original PassengerId column as row index 
all.set_index('PassengerId', inplace=True) 

# dropping Cabin & Name columns that were split previously 
all.drop(['Cabin', 'Name'], axis=1, inplace=True)

# checking all columns in our dataset match with those covered under column groupings
print(len(all.columns), len(numeric + ordinal + categorical) )

# Transforming all NA value types (pandas, numpy NA types, etc) into np.nan type for uniformity 
all = all.fillna(np.nan) 

# Splitting our merged dataset into original train and test sets, 
# where original Test Set had missing ('NA') values in Transported column 
X_train = all[all.Transported.notna()].drop('Transported', axis=1)
y_train = all[all.Transported.notna()].Transported.astype(int) 
X_test = all[all.Transported.isna()].drop('Transported', axis=1)

# deleting the target column name from our categorical columns grouping list 
categorical.remove('Transported')

# checking for data imbalance
print('Mean of target classes: ', y_train.mean())

''' Since this is a binary classification problem, calculating mean value of target column will suffice. 
Here it is 0.503 indicating that the data very well balanced into positive and negative target classes. 
In case we had severely unbalanced data between target classes (say 0.75:0.25), we may have to address 
the imbalance, for example, by bootstrapping the minority class column to match the sample size of the 
majority class, or alternatively sampling from majority class column to match sample size of minority class''' 


18 18
Mean of target classes:  0.5036236051995858


' Since this is a binary classification problem, calculating mean value of target column will suffice. \nHere it is 0.503 indicating that the data very well balanced into positive and negative target classes. \nIn case we had severely unbalanced data between target classes (say 0.75:0.25), we may have to address \nthe imbalance, for example, by bootstrapping the minority class column to match the sample size of the \nmajority class, or alternatively sampling from majority class column to match sample size of minority class'

In [13]:
# Using encoders on column groupings to convert data into numerical values where applicable 
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan) 

''' we will fit the encoders to training data and transform both training and test datasets using the 
same patterns learned from training data. This is to avoid leakage (of insights from test data (which 
is not available at the time of training in real world application) into training dataset) '''
X_train[ordinal + categorical] = ordinal_encoder.fit_transform(X_train[ordinal + categorical]) 
X_test[ordinal + categorical] = ordinal_encoder.transform(X_test[ordinal + categorical])

# check dtypes for each column and correct where necessary 
print(X_train.info(), y_train.info(), X_test.info(), sep = '\n\n')
print(X_train.describe(), y_train.describe(), X_test.describe(), sep = '\n\n')


<class 'pandas.core.frame.DataFrame'>
Index: 8693 entries, 0001_01 to 9280_02
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8492 non-null   float64
 1   CryoSleep     8476 non-null   float64
 2   Destination   8511 non-null   float64
 3   Age           8514 non-null   float64
 4   VIP           8490 non-null   float64
 5   RoomService   8512 non-null   float64
 6   FoodCourt     8510 non-null   float64
 7   ShoppingMall  8485 non-null   float64
 8   Spa           8510 non-null   float64
 9   VRDeck        8505 non-null   float64
 10  group         8693 non-null   int64  
 11  id            8693 non-null   int64  
 12  deck          8494 non-null   float64
 13  cabin_no      8494 non-null   float64
 14  side          8494 non-null   float64
 15  FName         8493 non-null   float64
 16  LName         8493 non-null   float64
dtypes: float64(15), int64(2)
memory usage: 1.4+ MB
<class 'pandas.core.

In [14]:
# calculate min & max values for all categorical features combined 
min_cat = X_train[categorical].min().min()
max_cat = X_train[categorical].max().max()
print('Min-Max spread of values in categorical columns: \n', min_cat, max_cat) 

# Scaling using MinMaxScaler 
mmScaler = MinMaxScaler(feature_range=(min_cat, max_cat)) 
X_train[numeric + ordinal] = mmScaler.fit_transform(X_train[numeric + ordinal]) 
X_test[numeric + ordinal] = mmScaler.transform(X_test[numeric + ordinal]) 

# checking column-wise standard deviation for scaled data 
print('column-wise standard deviation in Training set after min-max scaling')
print(X_train.std(axis=0).T)
print('column-wise standard deviation in Test set after min-max scaling')
print(X_test.std(axis=0).T)

Min-Max spread of values in categorical columns: 
 0.0 7.0
column-wise standard deviation in Training set after min-max scaling
HomePlanet      0.798155
CryoSleep       0.479531
Destination     0.820237
Age             1.283837
VIP             0.151303
RoomService     0.325750
FoodCourt       0.378373
ShoppingMall    0.180184
Spa             0.355094
VRDeck          0.332326
group           2.015002
id              1.054241
deck            1.778233
cabin_no        1.891801
side            0.500006
FName           1.982338
LName           2.026342
dtype: float64
column-wise standard deviation in Test set after min-max scaling
HomePlanet      0.811815
CryoSleep       0.482598
Destination     0.807489
Age             1.256373
VIP             0.131825
RoomService     0.296578
FoodCourt       0.358691
ShoppingMall    0.167110
Spa             0.348996
VRDeck          0.361702
group           2.049077
id              1.018221
deck            1.719892
cabin_no        1.903261
side            0

In [15]:
# Using KNN Imputer to impute missing values 

# ensure all NA values are represented uniformluy as np.nan 
X_train = X_train.fillna(np.nan)
X_test = X_test.fillna(np.nan) 

# Instantiate KNNImputer using default 5 neighbors 
imputer = KNNImputer(n_neighbors=5) 

'''KNN Imputer will return numpy arrays. In order to retain our dataframe with column names 
(to use for further preprocessig), we will assign the numpy output to our pandas dataframes 
using .loc[:] accessor for all data. '''

X_train.loc[:] = imputer.fit_transform(X_train) 
X_test.loc[:] = imputer.transform(X_test) 

# Checking if any missing values still remain 
print(X_train.isna().sum().T) 
print(X_test.isna().sum().T) 


/opt/anaconda3/envs/se411/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/opt/anaconda3/envs/se411/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/opt/anaconda3/envs/se411/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/opt/anaconda3/envs/se411/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/opt/anaconda3/envs/se411/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/opt/anaconda3/envs/se411/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
group           0
id              0
deck            0
cabin_no        0
side            0
FName           0
LName           0
dtype: int64
HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
group           0
id              0
deck            0
cabin_no        0
side            0
FName           0
LName           0
dtype: int64


In [16]:
# Calculating correlation (linear) and mutual-info (non-linear) of dataset columns / features with target column 
correlation = pd.concat([X_train, y_train], axis=1).corr().Transported.abs().values[:-1]
mutual_info = mutual_info_regression(X_train, y_train, random_state=SEED) 
associations = {'correlation': correlation, 'mutual_info': mutual_info} 
associations = pd.DataFrame(associations, index=X_train.columns).sort_values(by='correlation', ascending=False)
display(associations.style.background_gradient() )

# Checking summary statistics after missing value imputation 
print('Summary statistics after missing value imputation: ') 
print(X_train.info(), X_train.describe().T, sep='\n') 
print(X_test.info(), X_test.describe().T, sep='\n') 

,correlation,mutual_info
CryoSleep,0.464822,0.122334
RoomService,0.243517,0.064168
Spa,0.218882,0.080261
VRDeck,0.206177,0.062403
HomePlanet,0.119572,0.021771
deck,0.117175,0.028820
Destination,0.109219,0.016185
side,0.103062,0.017440
Age,0.072878,0.017640
id,0.066390,0.004860


Summary statistics after missing value imputation: 
<class 'pandas.core.frame.DataFrame'>
Index: 8693 entries, 0001_01 to 9280_02
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8693 non-null   float64
 1   CryoSleep     8693 non-null   float64
 2   Destination   8693 non-null   float64
 3   Age           8693 non-null   float64
 4   VIP           8693 non-null   float64
 5   RoomService   8693 non-null   float64
 6   FoodCourt     8693 non-null   float64
 7   ShoppingMall  8693 non-null   float64
 8   Spa           8693 non-null   float64
 9   VRDeck        8693 non-null   float64
 10  group         8693 non-null   float64
 11  id            8693 non-null   float64
 12  deck          8693 non-null   float64
 13  cabin_no      8693 non-null   float64
 14  side          8693 non-null   float64
 15  FName         8693 non-null   float64
 16  LName         8693 non-null   float64
dtypes: float64(17)


In [ ]:
# Transforming Categorical features using OneHotEncoder 
print('Before OHE', X_train.shape, X_test.shape, sep='\n')
onehot = OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False) 
onehot = make_column_transformer((onehot, categorical), remainder='passthrough')
X_train = onehot.fit_transform(X_train)
X_test = onehot.transform(X_test) 
print('After OHE', X_train.shape, X_test.shape, sep='\n')


# scaler = StandardScaler(with_mean=True)
scaler = RobustScaler(with_centering=True)
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test) 

# Retaining full training set for training model to be used for submission  
X_train_full, y_train_full = X_train.copy(), y_train.copy()

# Creating training and validation split (including corresponding indices from the base set) to evaluate model performances 
X_train, X_val, y_train, y_val, tts_tr, tts_val = train_test_split(X_train_full, y_train_full, np.arange(X_train_full.shape[0]), 
                                                                   test_size=0.20, stratify=y_train_full, random_state=SEED) 
print([i.shape[0] for i in [X_train, X_val, y_train, y_val, tts_tr, tts_val] ]) 
print('SEED: ', SEED)

Before OHE
(8693, 17)
(4277, 17)
After OHE
(8693, 69)
(4277, 69)
[6954, 1739, 6954, 1739, 6954, 1739]
SEED:  50309


/opt/anaconda3/envs/se411/lib/python3.13/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [18]:

pred_dict = dict() # to save separate predictions basis shuffled splits of training data 
n_splits = 5 # no of shuffled splits of data we will try 

In [19]:
# Tuned GradientBoostingClassifier with early stopping basis validation set 
'''Minimizing over-fitting, improving model generalization while using maximum possible training data for fitting our model ''' 
sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=2, random_state=SEED, ) 

for i, (tr_ids, ts_ids) in enumerate(sss.split(X_train_full, y_train_full)): 
    # training on shuffled dataset with 0.2 validation_fraction 
    model = GradientBoostingClassifier(max_depth=7, max_features='sqrt', n_estimators=600, n_iter_no_change=50, tol=0, subsample=0.75, learning_rate=0.05, validation_fraction=0.2, verbose=0, random_state=SEED+i )
    model = model.fit(X_train_full[tr_ids], y_train_full[tr_ids] ) 
    # Checking OOB scores 
    raw_ests = model.n_estimators_ 
    opt_ests = np.argmax(np.cumsum(model.oob_improvement_)) + 1 
    estop_ests = raw_ests - model.n_iter_no_change 
#     print(opt_ests, raw_ests, sep=' / ' )
    print('Peak oob cum. improv: ', np.max(np.cumsum(model.oob_improvement_)), np.cumsum(model.oob_improvement_)[opt_ests-1] )
    print('Before n_est tuning', i, np.sum(model.oob_improvement_), sep=': ')
#     print(i, 'log_loss', log_loss(y_train_full[ts_ids], model.predict_proba(X_train_full[ts_ids])[:,1])) 
#     print(i, 'accuracy', model.score(X_train_full[ts_ids], y_train_full[ts_ids]), '\n') 
    
    ''' To refit the model using n_estimators corresponding to peak cum. improvement, use n_estimators=opt_ests '''
    
    # Refit the model using n_estimators = raw_ests - early stopping 
    model = GradientBoostingClassifier(max_depth=7, max_features='sqrt', n_estimators=estop_ests, n_iter_no_change=50, tol=0, subsample=0.75, learning_rate=0.05, validation_fraction=0.2, verbose=0, random_state=SEED+i) 
    model = model.fit(X_train_full[tr_ids], y_train_full[tr_ids]) 
    # Checking OOB scores 
    opt_ests = np.argmax(np.cumsum(model.oob_improvement_)) + 1 
#     print(opt_ests, model.n_estimators_, sep=' / ' )
    print('Peak oob cum. improv: ', np.max(np.cumsum(model.oob_improvement_)), np.cumsum(model.oob_improvement_)[opt_ests-1] )
    print('After n_est tuning', i, np.sum(model.oob_improvement_), sep=': ')
#     print(i, 'log_loss', log_loss(y_train_full[ts_ids], model.predict_proba(X_train_full[ts_ids])[:,1])) 
#     print(i, 'accuracy', model.score(X_train_full[ts_ids], y_train_full[ts_ids]), '\n') 

    # predict on test set using model trained on StratifiedShuffleSplit of training data with 0.2 validation_fraction 
    pred_dict['gbc_'+str(i)] = model.predict_proba(X_test)[:, 1] 
    
predict_frame = pd.DataFrame(pred_dict)
predict_frame['gb_avg'] = predict_frame.iloc[:,-n_splits:].mean(axis=1) # soft voting gives better results than hard voting 
# predict_frame['gb_avg'] = predict_frame.median(axis=1) # hard voting  
print(predict_frame.describe()) 

y_test_pred = np.round(predict_frame['gb_avg'].values)  
# y_test_pred = np.round(predict_frame[2].values)  
print(y_test_pred[:5]) 
print(np.mean(y_test_pred)) 

submission = pd.DataFrame(dict(PassengerId = test.PassengerId, Transported = y_test_pred)) 
submission.Transported = submission.Transported.astype('bool')
submission.to_csv('submission.csv', index=False)
submission.head() 

print('SEED:', SEED) # to see THE random seed used in this run 
print ( ((predict_frame > 0.45) & (predict_frame < 0.55)).sum() )
# print(model.__dict__) 


/var/folders/gr/pfv5w5hx4m904t97xs_9qv7m0000gn/T/ipykernel_84124/3152141321.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  model = model.fit(X_train_full[tr_ids], y_train_full[tr_ids] )


Peak oob cum. improv:  0.9832589663906399 0.9832589663906399
Before n_est tuning: 0: 0.9737011769586532


/var/folders/gr/pfv5w5hx4m904t97xs_9qv7m0000gn/T/ipykernel_84124/3152141321.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  model = model.fit(X_train_full[tr_ids], y_train_full[tr_ids])


Peak oob cum. improv:  0.9434832178930596 0.9434832178930596
After n_est tuning: 0: 0.9235796954983999


/var/folders/gr/pfv5w5hx4m904t97xs_9qv7m0000gn/T/ipykernel_84124/3152141321.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  model = model.fit(X_train_full[tr_ids], y_train_full[tr_ids] )


...

Peak oob cum. improv:  0.8509548324066896 0.8509548324066896
After n_est tuning: 3: 0.8414089459574485


/var/folders/gr/pfv5w5hx4m904t97xs_9qv7m0000gn/T/ipykernel_84124/3152141321.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  model = model.fit(X_train_full[tr_ids], y_train_full[tr_ids] )


Peak oob cum. improv:  0.9182285439050208 0.9182285439050208
Before n_est tuning: 4: 0.8925148352351902


/var/folders/gr/pfv5w5hx4m904t97xs_9qv7m0000gn/T/ipykernel_84124/3152141321.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  model = model.fit(X_train_full[tr_ids], y_train_full[tr_ids])


Peak oob cum. improv:  0.8571759911467728 0.8571759911467728
After n_est tuning: 4: 0.852780471214627
             gbc_0        gbc_1        gbc_2        gbc_3        gbc_4  \
count  4277.000000  4277.000000  4277.000000  4277.000000  4277.000000   
mean      0.504877     0.501223     0.504382     0.504965     0.502580   
std       0.355613     0.355872     0.352190     0.341037     0.345424   
min       0.001557     0.001232     0.004371     0.004417     0.001755   
25%       0.127034     0.126333     0.131439     0.145072     0.135698   
50%       0.510803     0.510591     0.519956     0.517604     0.515690   
75%       0.883004     0.883550     0.875220     0.854289     0.857246   
max       0.995431     0.995659     0.994472     0.990830     0.994375   

            gb_avg  
count  4277.000000  
mean      0.503606  
std       0.347406  
min       0.003065  
25%       0.135924  
50%       0.518557  
75%       0.864757  
max       0.993035  
[1. 0. 1. 1. 1.]
0.5155482815057283
SEED: 

In [25]:
# # Use this code block to generate a quick submission using any specified model 
set_random_seed = SEED 
y_test_pred = GradientBoostingClassifier(learning_rate=0.05, max_depth=7, max_features='log2', n_estimators=154, 
                                    subsample=0.55).fit(X_train_full, y_train_full).predict(X_test) # 0.80406 V15 
print(np.mean(y_test_pred)) 
submission = pd.DataFrame(dict(PassengerId = test.PassengerId, Transported = y_test_pred)) 
submission.Transported = submission.Transported.astype('bool')
submission.to_csv('submission.csv', index=False)
submission.head() 


0.514145429039046


,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
